In [22]:
import pandas as pd
import numpy as np

In [23]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.3f}'.format)

In [24]:
OUTPUT_DIR="../../output/openmpi/experimental"
OUTPUT_SEQ="../../output/openmp/experimental"

EXECUTIONS = 50
CITIES = [13, 14, 15]
PROCESSES = [1, 2, 3, 4, 5, 10]
ITERATIONS = 10

In [25]:
def geometric_mean_of_mean_of_instances(mean_of_instances):
    geometric_mean = mean_of_instances.prod()**(1.0/EXECUTIONS)
    return geometric_mean

def std_mean_of_mean_of_instances(mean_of_instances):
    # std_mean = np.std(mean_of_instances)/(EXECUTIONS**2)
    std_mean = np.std(mean_of_instances)
    return std_mean

def extract_time_stats_from_file(file_name):
    file = open(file_name)
    data = []
    for exec in range(1, EXECUTIONS + 1):
        data.append(float(file.readline().split(" ")[1]))
    np_array = np.array(data)
    file.close()
    return np.sum(np_array), np_array


In [26]:
df_time = pd.DataFrame(columns = ["Version", "Cities", "Processes", "Instance_mean", "Instance_std", "Instance_std_%(x100)", "Sum_mean", "Sum_std", "Sum_std_%(x100)"])

for city in list(CITIES):
    sum_array, array_of_instance_array = [], []
    for iteration in range(1,ITERATIONS + 1):
        file_name = OUTPUT_SEQ + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
        
        sum, instance_array = extract_time_stats_from_file(file_name)
        
        array_of_instance_array.append(instance_array)
        sum_array.append(sum)
    
    mean_of_instances = np.mean(array_of_instance_array, axis=0)
    
    geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
    std_mean = std_mean_of_mean_of_instances(mean_of_instances)
    list_row = ["Sequential", city, "1", geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
    df_time.loc[len(df_time)] = list_row
            
    for process in list(PROCESSES):
        sum_array, array_of_instance_array = [], []
        for iteration in range(1,ITERATIONS + 1):
            file_name = OUTPUT_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(process) + "_processes_result.out"
            
            sum, instance_array = extract_time_stats_from_file(file_name)

            array_of_instance_array.append(instance_array)
            sum_array.append(sum)

        mean_of_instances = np.mean(array_of_instance_array, axis=0)
        
        geometric_mean = geometric_mean_of_mean_of_instances(mean_of_instances)
        std_mean = std_mean_of_mean_of_instances(mean_of_instances)
        list_row = ["Parallel", city, process, geometric_mean, std_mean, std_mean/geometric_mean*100, np.mean(sum_array), np.std(sum_array), np.std(sum_array)/np.mean(sum_array)*100]
        df_time.loc[len(df_time)] = list_row
df_time

,Version,Cities,Processes,Instance_mean,Instance_std,Instance_std_%(x100),Sum_mean,Sum_std,Sum_std_%(x100)
0,Sequential,13,1,0.382,0.397,103.981,24.774,0.187,0.757
1,Parallel,13,1,0.427,0.419,98.148,27.170,0.215,0.790
2,Parallel,13,2,0.218,0.211,96.722,13.819,0.082,0.596
3,Parallel,13,3,0.150,0.144,96.117,9.478,0.046,0.483
4,Parallel,13,4,0.116,0.110,94.874,7.321,0.024,0.324
5,Parallel,13,5,0.097,0.090,93.433,6.050,0.018,0.290
6,Parallel,13,10,0.075,0.065,86.666,4.623,0.116,2.507
7,Sequential,14,1,1.539,3.838,249.376,129.097,0.391,0.303
8,Parallel,14,1,1.679,3.995,237.882,137.736,1.083,0.786
9,Parallel,14,2,0.860,2.010,233.767,70.018,0.351,0.502


Speedup and Efficiency

In [28]:
df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Processes", "Instance_speedup", "Instance_efficiency", "Sum_speedup", "Sum_efficiency"])
# df_speedup_efficiency = pd.DataFrame(columns = ["Cities", "Processes", "Sum Speedup", "Sum Efficiency"])

df_sequential = df_time[df_time['Version'].isin(["Sequential"])]
df_parallel = df_time[df_time['Version'].isin(["Parallel"])]

for city in list(CITIES):
    df_seq = df_sequential[df_sequential["Cities"].isin([city])]
    for process in list(PROCESSES):
        df_par = df_parallel[df_parallel["Cities"].isin([city]) & df_parallel["Processes"].isin([process])]
        speedup_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()
        efficiency_instance = df_seq["Instance_mean"].item()/df_par["Instance_mean"].item()/float(process)
        speedup_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()
        efficiency_sum = df_seq["Sum_mean"].item()/df_par["Sum_mean"].item()/float(process)

        list_row = [city, process, speedup_instance, efficiency_instance, speedup_sum, efficiency_sum]
        # list_row = [city, process, speedup_sum, efficiency_sum]
        df_speedup_efficiency.loc[len(df_speedup_efficiency)] = list_row

df_speedup_efficiency["Processes"] = df_speedup_efficiency["Processes"].astype(int)
df_speedup_efficiency["Cities"] = df_speedup_efficiency["Cities"].astype(int)
df_speedup_efficiency

,Cities,Processes,Instance_speedup,Instance_efficiency,Sum_speedup,Sum_efficiency
0,13,1,0.895,0.895,0.912,0.912
1,13,2,1.750,0.875,1.793,0.896
2,13,3,2.547,0.849,2.614,0.871
3,13,4,3.283,0.821,3.384,0.846
4,13,5,3.952,0.790,4.095,0.819
5,13,10,5.069,0.507,5.359,0.536
6,14,1,0.916,0.916,0.937,0.937
7,14,2,1.790,0.895,1.844,0.922
8,14,3,2.605,0.868,2.703,0.901
9,14,4,3.344,0.836,3.507,0.877


Output 1

In [41]:
# translate to portuguese
df_time.loc[df_time["Version"] == "Parallel", "Version"] = "Paralelo"
df_time.loc[df_time["Version"] == "Sequential", "Version"] = "Sequencial"
res = df_time.rename(columns={
    "Version": "Versão",
    "Processes": "Processos",
    "Cities": "# Cidades",
    "Instance_mean": "Média geométrica instância",
    "Instance_std": "Desvio padrão instância",
    "Instance_std_%(x100)": "% Desvio padrão instância (x100)",
    "Sum_mean": "Média aritmética tempo total",
    "Sum_std": "Desvio padrão tempo total",
    "Sum_std_%(x100)": "% Desvio padrão tempo total (x100)"
})
res

,Versão,# Cidades,Processos,Média geométrica instância,Desvio padrão instância,% Desvio padrão instância (x100),Média aritmética tempo total,Desvio padrão tempo total,% Desvio padrão tempo total (x100)
0,Sequencial,13,1,0.382,0.397,103.981,24.774,0.187,0.757
1,Paralelo,13,1,0.427,0.419,98.148,27.170,0.215,0.790
2,Paralelo,13,2,0.218,0.211,96.722,13.819,0.082,0.596
3,Paralelo,13,3,0.150,0.144,96.117,9.478,0.046,0.483
4,Paralelo,13,4,0.116,0.110,94.874,7.321,0.024,0.324
5,Paralelo,13,5,0.097,0.090,93.433,6.050,0.018,0.290
6,Paralelo,13,10,0.075,0.065,86.666,4.623,0.116,2.507
7,Sequencial,14,1,1.539,3.838,249.376,129.097,0.391,0.303
8,Paralelo,14,1,1.679,3.995,237.882,137.736,1.083,0.786
9,Paralelo,14,2,0.860,2.010,233.767,70.018,0.351,0.502


In [42]:
res1 = res.pivot(index=["Processos", "Versão"], columns="# Cidades", values=["Média geométrica instância", "Desvio padrão instância", "% Desvio padrão instância (x100)"])
res2 = res.pivot(index=["Processos", "Versão"], columns="# Cidades", values=["Média aritmética tempo total", "Desvio padrão tempo total", "% Desvio padrão tempo total (x100)"])
res1 = res1.sort_values(by='Versão', ascending=False)
res2 = res2.sort_values(by='Versão', ascending=False)
res1

Média geométrica instância              \
# Cidades                                    13    14    15   
Processos Versão                                              
1         Sequencial                      0.382 1.539 7.627   
          Paralelo                        0.427 1.679 8.122   
2         Paralelo                        0.218 0.860 4.189   
3         Paralelo                        0.150 0.591 2.888   
4         Paralelo                        0.116 0.460 2.260   
5         Paralelo                        0.097 0.382 1.888   
10        Paralelo                        0.075 0.299 1.533   

                     Desvio padrão instância              \
# Cidades                                 13    14    15   
Processos Versão                                           
1         Sequencial                   0.397 3.838 9.265   
          Paralelo                     0.419 3.995 9.674   
2         Paralelo                     0.211 2.010 4.909   
3         Paralelo                     0.144 1.354 3.329   
4         Paralelo                     0.110 1.028 2.564   
5         Paralelo                     0.090 0.834 2.101   
10        Paralelo                     0.065 0.583 1.559   

                     % Desvio padrão instância (x100)                  
# Cidades                                          13      14      15  
Processos Versão                                                       
1         Sequencial                          103.981 249.376 121.476  
          Paralelo                             98.148 237.882 119.111  
2         Paralelo                             96.722 233.767 117.183  
3         Paralelo                             96.117 229.130 115.265  
4         Paralelo                             94.874 223.432 113.449  
5         Paralelo                             93.433 218.507 111.235  
10        Paralelo                             86.666 195.326 101.719

In [43]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [44]:
res1

In [45]:
res2

Output 2

In [47]:
# translate to portuguese
res = df_speedup_efficiency.rename(columns={"Cities": "# Cidades", "Processes": "Processos", "Instance_speedup": "Speedup instância", "Instance_efficiency": "Eficiência instância", "Sum_speedup": "Speedup tempo total", "Sum_efficiency": "Eficiência tempo total"})
res

,# Cidades,Processos,Speedup instância,Eficiência instância,Speedup tempo total,Eficiência tempo total
0,13,1,0.895,0.895,0.912,0.912
1,13,2,1.750,0.875,1.793,0.896
2,13,3,2.547,0.849,2.614,0.871
3,13,4,3.283,0.821,3.384,0.846
4,13,5,3.952,0.790,4.095,0.819
5,13,10,5.069,0.507,5.359,0.536
6,14,1,0.916,0.916,0.937,0.937
7,14,2,1.790,0.895,1.844,0.922
8,14,3,2.605,0.868,2.703,0.901
9,14,4,3.344,0.836,3.507,0.877


In [48]:
res1 = res.pivot(index="Processos", columns="# Cidades", values=["Speedup instância", "Eficiência instância"])
res2 = res.pivot(index="Processos", columns="# Cidades", values=["Speedup tempo total", "Eficiência tempo total"])
res1

Speedup instância             Eficiência instância            
# Cidades                13    14    15                   13    14    15
Processos                                                               
1                     0.895 0.916 0.939                0.895 0.916 0.939
2                     1.750 1.790 1.821                0.875 0.895 0.910
3                     2.547 2.605 2.641                0.849 0.868 0.880
4                     3.283 3.344 3.375                0.821 0.836 0.844
5                     3.952 4.031 4.039                0.790 0.806 0.808
10                    5.069 5.152 4.976                0.507 0.515 0.498

In [49]:
res1 = res1.T.swaplevel(1)
res1 = res1.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)
res2 = res2.T.swaplevel(1)
res2 = res2.style.set_table_styles([dict(selector='th', props=[('text-align', 'center')])]).format(precision=3)

In [50]:
res1

In [51]:
res2

Compare time dedicated to the sequential portion of the algorithms with total time

In [52]:
OUTPUT_EXP_DIR="../../output/openmpi/experimental"
OUTPUT_THE_DIR="../../output/openmpi/theoretical"
CITIES = [15]
PROCESSES = [5]
ITERATIONS = 10
EXECUTIONS = 50

In [53]:
# for city in list(CITIES):
#     sum_experimental, sum_theoretical = [], []
#     for iteration in range(1,ITERATIONS + 1):
#         file_name = OUTPUT_EXP_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
#         sum, instance_array = extract_time_stats_from_file(file_name)
#         sum_experimental.append(sum)
        
#         file_name = OUTPUT_THE_DIR + "/" + str(iteration) + "_seq_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_result.out"
#         sum, instance_array = extract_time_stats_from_file(file_name)
#         sum_theoretical.append(sum)
#     print("Sequential portion analysis - Sequential {} cities: {} ([0, 1])".format(city, np.mean(sum_theoretical)/np.mean(sum_experimental)))

            
#     for process in list(PROCESSES):
#         sum_experimental, sum_theoretical = [], []
#         for iteration in range(1,ITERATIONS + 1):
#             file_name = OUTPUT_EXP_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(process) + "_processes_result.out"
#             sum, instance_array = extract_time_stats_from_file(file_name)
#             sum_experimental.append(sum)
            
#             file_name = OUTPUT_THE_DIR + "/" + str(iteration) + "_par_" + str(city) + "_cities_" + str(EXECUTIONS) + "_executions_" + str(process) + "_processes_result.out"
#             sum, instance_array = extract_time_stats_from_file(file_name)
#             sum_theoretical.append(sum)
#         print("Sequential portion analysis - Parallel {} cities {} processes: {} ([0, 1])".format(city, process, np.mean(sum_theoretical)/np.mean(sum_experimental)))

        